In [6]:
import os
import datetime
import astromatic_wrapper as aw

# Get the current date and time to
log_path = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_test')

# Set defaults
base_path = '/Users/lasilla/LaSillaRepo/stacking/'
paths = {
    'temp': os.path.join(base_path, 'temp'),
    'log': os.path.join(base_path, 'log', log_path),
    'config': os.path.join(base_path, 'config'),
    'catalogs': os.path.join(base_path, 'catalogs'),
    'stacks': os.path.join(base_path, 'stacks'),
    'images': '/Users/lasilla/dfosc_testdata/reduced/'
}

# If you are using a system wide install you can remove the following code
# that specifies build_paths for each code
build_path = '/Users/lasilla/opt/anaconda2/envs/astromatic'
build_paths = {
    'SExtractor': os.path.join(build_path, 'bin', 'sex'),
    'SCAMP': os.path.join(build_path, 'bin', 'scamp'),
    'SWARP': os.path.join(build_path, 'bin', 'swarp'),
    'PSFEx': os.path.join(build_path, 'bin', 'psfex')
}
# Create the pipeline
pipeline = aw.utils.pipeline.Pipeline(paths, build_paths=build_paths)

'/Users/lasilla/LaSillaRepo/stacking/log/2020-02-10_23-07-06_test' does not exist, create (y/n)?y


In [ ]:
# some properties of the dfosc observations
gain = 0.164 * u.electron / u.adu
ron = 4 * u.electron
exptime = 10*u.second
gain_scaled = 

In [ ]:
# set-up the pipeline
def build_pipeline(pipeline, exposures,
                   ref_catalog='UCAC-4',
                   ref_band = 'DEFAULT',
                   frames = [],
                   stack_name = 'testwasp24_stacked.fits',
                   output_cat_name =   'testwasp24.ldac.fits'):
    # Generate catalogs from SExtractor
    catalog_names = []
    for files in exposures:
        # create names for the output catalogs for each image
        catalog_names.append(os.path.join(pipeline.paths['temp'],
                                          os.path.basename(files['image']).replace('.fits',
                                                                                   '.ldac')))
        kwargs  = {
            # image to SExtract
            'files': files,
            'api_kwargs':{
                # arguments to initialize Astromatic class
                'config': {'CATALOG_NAME': catalog_names[-1],
                           'CATALOG_TYPE': 'FITS_LDAC',
                           'DETECT_THRESH':'5.',
                           'ANALYSIS_THRESH':'5.',
                           'FILTER': 'N',
                           'GAIN': '3.3',
                           # as the exposures are the "pure counts"
                           'PARAMETERS_NAME':'default.param'},
                # frames to run Sextractor on
                'frames': frames,
                'config_file': os.path.join(pipeline.paths['config'],
                                            'SCAM_OIII_par.sex')
            }
        }
        # Add the step to the pipeline
        pipeline.add_step(aw.api.run_sex, ['step1', 'SExtractor'],
                          ignore_errors=True, ignore_exceptions=True,
                          **kwargs)

    # Get astrometric solution from SCAMP
    # Use SCAMP to get astrometric solutions
    kwargs = {
        'catalogs': catalog_names,
        'api_kwargs': {
            'config': {
                'ASTREF_CATALOG': ref_catalog,
                'ASTREF_BAND': ref_band,
                'CATALOG_TYPE': 'FITS_LDAC',
                'SOLVE_PHOTOM': 'N',
                'CHECKPLOT_DEV': 'NULL',
                'NTHREADS':'2'
            },
            'config_file': os.path.join(pipeline.paths['config'],
                                        'default.scamp')


        }
    }
    pipeline.add_step(aw.api.run_scamp, ['step2', 'SCAMP'],**kwargs)

    # Resample (rotate and scale) and combine (stack) images
    stack_filename = os.path.join(pipeline.paths['temp'], stack_name)
    kwargs = {
        'filenames': [exp['image'] for exp in exposures],
        'api_kwargs': {
            'config': {
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'WEIGHT_SUFFIX': '.wtmap.fits',
                'IMAGEOUT_NAME': stack_filename,
                'WEIGHTOUT_NAME': stack_filename.replace('.fits','.wtmap.fits'),
                'RESAMPLE_DIR': pipeline.paths['temp'],
            }
        },
        'frames': frames
    }
    pipeline.add_step(aw.api.run_swarp, ['step3', 'SWarp'], **kwargs)

    # Get positions in stack for PSF photometry
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': False,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'NTHREADS':'2',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE',
                       'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE',
                        'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                        'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT',
                        'FLUX_RADIUS', 'ELONGATION',
                        'VIGNET(20,20)', 'SNR_WIN'],
        },
        'frames': frames
    }
    pipeline.add_step(aw.api.run_sex, ['step4', 'SExtractor'], **kwargs)

    # Calculate PSF
    kwargs = {
        'catalogs': stack_filename.replace('.fits', '.cat'),
        'api_kwargs': {
            'config': {
                'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'CHECKPLOT_DEV': 'NULL',
                'PSF_SUFFIX': '.psf'
            },
        }
    }
    pipeline.add_step(aw.api.run_psfex, ['step5', 'PSFEx'], **kwargs)

    # Calculate PSF photometry for stacked image
    # Get positions in stack for PSF photometry
    catalog_name = os.path.join(pipeline.paths['catalogs'], output_cat_name)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'PSF_NAME': os.path.join(stack_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': False,
                'CATALOG_NAME': catalog_name,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'NTHREADS':'2',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE',
                       'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                       'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE',
                       'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                       'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                       'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY',
                       'DELTAPSF_SKY','ERRX2PSF_WORLD','ERRY2PSF_WORLD',
                       'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF'],
        },
        'frames': frames
    }
    pipeline.add_step(aw.api.run_sex, ['step6', 'SExtractor'], **kwargs)

    def save_output(pipeline, old_stack, new_stack, old_cat, new_cat):
            # Copy the final stack and catalog from the temp folder
            import shutil
            # Move the weight map if it exists
            if os.path.isfile(old_stack.replace('.fits', '.wtmap.fits')):
                shutil.move(old_stack.replace('.fits', '.wtmap.fits'),
                            new_stack.replace('.fits', '.wtmap.fits'))
            shutil.move(old_stack, new_stack)
            shutil.move(old_cat, new_cat)
            result = {
                'status': 'success'
            }
            return result

    kwargs = {
        'old_stack': stack_filename,
        'new_stack': os.path.join(pipeline.paths['stacks'], stack_name),
        'old_cat': catalog_name,
        'new_cat': os.path.join(pipeline.paths['catalogs'], output_cat_name)
    }
    pipeline.add_step(save_output, ['step7', 'save_output'], **kwargs)

    return pipeline
